## Text Modelling

We have downloaded some speeches by B. Obama and M. Romney made during US Presidential election 2012. The speeches are uploaded on GitHUB and we would try to find some imortant topics each of their speeches consisted of. We will do Topic modelling using Gensim and visualize these topics using a womderful package called pyLDAvis.

**NOTE:** If pyLDAvis visualization diagrams are NOT visible in this notebook, refer to this jupyter nbviewer version https://nbviewer.jupyter.org/github/Rajesh-Nair/Text-modelling-on-speeches/blob/master/Text%20Modelling%20using%20gensim%20%28LSI%20and%20LDA%29.ipynb

In [1]:
# Find folders and text under a given path
import os

# For dataframe/tables
import pandas as pd

# NLTK package
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Gensim
from gensim import corpora
from gensim import models

# Visualization
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

C:\Users\acer\Anaconda3\envs\py35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Text document folder
folder = 'speeches'

# Stop words set
ignorewords = set(stopwords.words("english"))

# Lemmatizer
lemmatizer = WordNetLemmatizer().lemmatize

#define dictionary  and corpus
dictionary = corpora.Dictionary()
corpus = list()

# Clean text
# 1. Word Tokenize text
# 2. Ignore stopwords and non alpha characters, words less than 4 in length
# 3. conver to lower case
# 4. Lemmatize the words
# 5. Update/Add to Dictionary
# 6. Create Bag of Words
def clean_corpus(text) :
    textlist = list()
    textlist.append([lemmatizer(word.lower()) for word in word_tokenize(text) \
                       if word not in ignorewords and word.isalpha() and len(word) > 4 ])
    dictionary.add_documents(textlist)
    for text in textlist :
        corpus.append(dictionary.doc2bow(text, allow_update=True))    
    return(None)



In [3]:
# Read, clean and convert the speeches in to Tf-idf
corpus_tfidf = dict(); corpus_dict = dict()
for fld in  os.listdir(folder) :
    corpus = [] # Re-Initialize
    dictionary = corpora.Dictionary() # Re-Initialize
    for file in os.listdir(folder + '\\' + fld) :
        txtfile = folder + '\\' + fld + '\\' + file
        with open(txtfile, 'r',errors='ignore') as fobj :        
            clean_corpus(fobj.read())       
    # Tf-idf model
    Tfidf = models.TfidfModel(corpus, smartirs='ntc')
    corpus_tfidf[fld] = Tfidf[corpus]
    corpus_dict[fld] = dictionary

In [4]:
# LSI model
lsi = dict()
num_topics = 5 
num_words = 15

for fld in corpus_tfidf.keys() :
    lsi[fld] = models.LsiModel(corpus_tfidf[fld], id2word=corpus_dict[fld], num_topics=num_topics)


In [5]:
# Display the topics and words associated
for fld in corpus_tfidf.keys() :    
    print('*'*80)
    print(fld.upper())
    print(pd.DataFrame({"Topic "+ str(i+1):[topic[0] for topic in lsi[fld].show_topic(i,num_words)] \
                                    for i in range(num_topics)}, index = [i+1 for i in range(num_words)] ))

********************************************************************************
ROMNEY
    Topic 1     Topic 2       Topic 3     Topic 4       Topic 5
1   liberal       found         reply       peace      applause
2     found   searching        cancel      regime       dreamer
3    crisis   requested         leave        ally         plant
4   program     perhaps        falcon     dreamer        saving
5    school     nothing         berta      saving      remember
6    budget       could       library      weapon        little
7     going     liberal        valley    applause  unidentified
8    little       going      provided     veteran      audience
9    dollar      crisis         tweet       plant         going
10   nation      budget          view     iranian     attacking
11  percent       reply  presidential       build    imaginable
12    build    strategy      applause        wage         meant
13  company      little        ronald  capability    unfairness
14   always  str

In [6]:
# LDA model
lda = dict()
num_topics = 5 
num_words = 15

for fld in corpus_tfidf.keys() :
    lda[fld] = models.LdaModel(corpus_tfidf[fld], id2word=corpus_dict[fld], num_topics=num_topics)


In [7]:
# Display the topics and words associated
for fld in corpus_tfidf.keys() :  
    print('*'*80)
    print(fld.upper())
    print(pd.DataFrame({"Topic "+ str(i+1):[topic[0] for topic in lda[fld].show_topic(i,num_words)] \
                                    for i in range(num_topics)}, index = [i+1 for i in range(num_words)] ))

********************************************************************************
ROMNEY
         Topic 1       Topic 2     Topic 3         Topic 4    Topic 5
1   unidentified         reply       reply        applause      found
2         crisis        cancel     comment        audience  searching
3        teacher         legal       found         liberal  requested
4       remember         leave      closed           going    perhaps
5        student        income       berta         percent   applause
6          build     immigrant      falcon          school    liberal
7         church        growth      cancel        promised    library
8    scholarship       benefit   searching         assault      kerry
9         parent       program   requested         founder     valley
10        budget     encourage       marry          agenda   provided
11       program     delivered       leave     inspiration    nothing
12        school  unidentified     liberal  administration     crisis
13

In [8]:
# Visualize - LDA using pyLDAvis
vis = dict()
for fld in corpus_tfidf.keys() :
    vis[fld] = pyLDAvis.gensim.prepare(lda[fld],corpus=corpus_tfidf[fld],dictionary=corpus_dict[fld])

C:\Users\acer\Anaconda3\envs\py35\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


In [9]:
# Print topics in Romney's speeches
print(list(corpus_tfidf.keys())[0])
vis[list(corpus_tfidf.keys())[0]]

romney


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      31.432741        1       1 -0.003746  0.001585
2      20.721272        1       2 -0.000798 -0.003618
0      19.991714        1       3  0.001772  0.000906
3      18.105421        1       4  0.001318  0.001428
1       9.748853        1       5  0.001454 -0.000301, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
2041  Default  0.000000         reply  0.000000  30.0000  30.0000
6183  Default  0.000000         found  0.000000  29.0000  29.0000
5252  Default  0.000000     searching  0.000000  28.0000  28.0000
5423  Default  0.000000        cancel  0.000000  27.0000  27.0000
1770  Default  0.000000     requested  0.000000  26.0000  26.0000
3864  Default  0.000000         legal  0.000000  25.0000  25.0000
5022  Default  0.000000  unidentified  0.000000  24.0000  24.0000
4320  Default  0.000000       comment  0.000000  23.0000  23.0000
2881  Default  0.000000         berta  0.000000  22.0000  22.0000
240   Default  0.000000        falcon  0.000000  21.0000  21.0000
5478  Default  0.000000      applause  0.000000  20.0000  20.0000
4071  Default  0.000000        closed  0.000000  19.0000  19.0000
5972  Default  0.000000        income  0.000000  18.0000  18.0000
5941  Default  0.000000         leave  0.000000  17.0000  17.0000
710   Default  0.000000      audience  0.000000  16.0000  16.0000
2841  Default  0.000000   commonsense  0.000000  15.0000  15.0000
484   Default  0.000000         sheet  0.000000  14.0000  14.0000
5062  Default  0.000000     immigrant  0.000000  13.0000  13.0000
329   Default  0.000000       legally  0.000000  12.0000  12.0000
3077  Default  0.000000       percent  0.000000  11.0000  11.0000
3820  Default  0.000000     encourage  0.000000  10.0000  10.0000
2058  Default  0.000000        broken  0.000000   9.0000   9.0000
4628  Default  0.000000        martha  0.000000   8.0000   8.0000
4055  Default  0.000000     delivered  0.000000   7.0000   7.0000
3098  Default  0.000000       norfolk  0.000000   6.0000   6.0000
3095  Default  0.000000   grandfather  0.000000   5.0000   5.0000
1775  Default  0.000000    strengthen  0.000000   4.0000   4.0000
5112  Default  0.000000        reduce  0.000000   3.0000   3.0000
2585  Default  0.000000        growth  0.000000   2.0000   2.0000
4307  Default  0.000000     permanent  0.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
2321   Topic5  0.021365       diploma  0.141480   0.4376  -8.4049
4368   Topic5  0.021295      eligible  0.141544   0.4339  -8.4082
4114   Topic5  0.024832        staple  0.165275   0.4325  -8.2546
516    Topic5  0.021264        peaked  0.141573   0.4322  -8.4097
1492   Topic5  0.021246     illegally  0.141591   0.4313  -8.4105
2225   Topic5  0.022633       combine  0.151310   0.4281  -8.3473
1124   Topic5  0.021155        risked  0.141673   0.4264  -8.4148
448    Topic5  0.025134        fiscal  0.168501   0.4253  -8.2425
4307   Topic5  0.025624     permanent  0.173987   0.4126  -8.2232
5941   Topic5  0.035223         leave  0.248318   0.3750  -7.9050
2058   Topic5  0.027185        broken  0.187243   0.3983  -8.1641
1822   Topic5  0.025364        bright  0.174720   0.3982  -8.2334
2585   Topic5  0.033439        growth  0.251676   0.3096  -7.9570
5651   Topic5  0.025245   alternative  0.176139   0.3854  -8.2381
2618   Topic5  0.030508       benefit  0.230146   0.3073  -8.0487
358    Topic5  0.028138       balance  0.206612   0.3343  -8.1296
5022   Topic5  0.029017  unidentified  0.222105   0.2928  -8.0988
581    Topic5  0.027603       illegal  0.206959   0.3134  -8.1488
4960   Topic5  0.027517         track  0.211960   0.2864  -8.1519
3961   Topic5  0.030434       program  0.285105   0.0907  -8.0511
3077   Topic5  0.028986       percent  0.256498   0.1477  -8.0999
662    Topic5  

In [10]:
# Print topics in Obama's speeches
print(list(corpus_tfidf.keys())[1])
vis[list(corpus_tfidf.keys())[1]]

obama


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      31.625620        1       1 -0.004150  0.000778
1      27.495411        1       2  0.001756 -0.003603
2      16.166296        1       3 -0.000959 -0.001391
3      12.809822        1       4  0.002778  0.002632
0      11.902850        1       5  0.000576  0.001583, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
4739  Default  0.000000         kerry  0.000000  30.0000  30.0000
678   Default  0.000000       illegal  0.000000  29.0000  29.0000
3076  Default  0.000000        credit  0.000000  28.0000  28.0000
1431  Default  0.000000      employer  0.000000  27.0000  27.0000
1009  Default  0.000000     immigrant  0.000000  26.0000  26.0000
6266  Default  0.000000     stevenson  0.000000  25.0000  25.0000
6208  Default  0.000000  undocumented  0.000000  24.0000  24.0000
460   Default  0.000000     confirmed  0.000000  23.0000  23.0000
3102  Default  0.000000        weapon  0.000000  22.0000  22.0000
1428  Default  0.000000     judiciary  0.000000  21.0000  21.0000
3350  Default  0.000000       examine  0.000000  20.0000  20.0000
6443  Default  0.000000         cairo  0.000000  19.0000  19.0000
426   Default  0.000000       balance  0.000000  18.0000  18.0000
612   Default  0.000000   enforcement  0.000000  17.0000  17.0000
990   Default  0.000000        record  0.000000  16.0000  16.0000
3906  Default  0.000000       payment  0.000000  15.0000  15.0000
3932  Default  0.000000   guestworker  0.000000  14.0000  14.0000
4095  Default  0.000000        advice  0.000000  13.0000  13.0000
1985  Default  0.000000      lobbyist  0.000000  12.0000  12.0000
2172  Default  0.000000         labor  0.000000  11.0000  11.0000
4330  Default  0.000000       consent  0.000000  10.0000  10.0000
6787  Default  0.000000  consistently  0.000000   9.0000   9.0000
6859  Default  0.000000       chamber  0.000000   8.0000   8.0000
6453  Default  0.000000      facility  0.000000   7.0000   7.0000
2099  Default  0.000000  constitution  0.000000   6.0000   6.0000
4574  Default  0.000000      epilepsy  0.000000   5.0000   5.0000
1615  Default  0.000000        search  0.000000   4.0000   4.0000
3210  Default  0.000000    organizing  0.000000   3.0000   3.0000
4672  Default  0.000000         legal  0.000000   2.0000   2.0000
7221  Default  0.000000        income  0.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
2375   Topic5  0.038791        joseph  0.185814   0.5618  -8.4032
5816   Topic5  0.039018        invade  0.187226   0.5601  -8.3973
3653   Topic5  0.042117        ethnic  0.203101   0.5551  -8.3209
2172   Topic5  0.051610         labor  0.249232   0.5537  -8.1176
6208   Topic5  0.069963  undocumented  0.339676   0.5484  -7.8134
5437   Topic5  0.035897       ongoing  0.174446   0.5474  -8.4807
6960   Topic5  0.042085       penalty  0.204541   0.5473  -8.3217
6529   Topic5  0.036813         moses  0.179050   0.5466  -8.4555
3210   Topic5  0.048862    organizing  0.239028   0.5408  -8.1724
5264   Topic5  0.035527      compound  0.174067   0.5392  -8.4911
4904   Topic5  0.046083   destruction  0.227049   0.5337  -8.2309
3778   Topic5  0.046542      document  0.230568   0.5282  -8.2210
4672   Topic5  0.057782         legal  0.292208   0.5076  -8.0047
5728   Topic5  0.049911    compromise  0.253913   0.5016  -8.1511
2367   Topic5  0.048949      proposed  0.248930   0.5020  -8.1706
3794   Topic5  0.056396        border  0.304018   0.4437  -8.0290
5429   Topic5  0.075449        worker  0.493864   0.2496  -7.7379
2980   Topic5  0.048419   citizenship  0.255623   0.4646  -8.1815
1985   Topic5  0.064366      lobbyist  0.426201   0.2381  -7.8968
272    Topic5  0.050178    department  0.278903   0.4131  -8.1458
3837   Topic5  0.048215      abramoff  0.262452   0.4340  -8.1857
7387   Topic5  